In [1]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("List available databases and tables").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session

spark.catalog.listDatabases()

[Database(name='2022_10_22', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='2023_04_01', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse')]

In [2]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark.sql("USE 2023_04_01")
    # Assuming that 'ticker' is a valid Python variable

# query = f"""SELECT ticker_region FROM sym_ticker_region WHERE ticker_region LIKE "%-US" """
# df = spark.sql(query)
# df = df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
# ticker_list = df.collect()
# print(len(ticker_list))


DataFrame[]

In [41]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import when
import pyspark.pandas as ps
import numpy as np


#fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))

def query(ticker,date):
    query = f"""SELECT d.ticker_region, a.date, FF_PRICE_CLOSE_FP
                FROM FF_BASIC_AF a 
                LEFT JOIN sym_ticker_region d ON d.fsym_id = a.fsym_id 
                WHERE d.ticker_region = "{ticker}-US" AND a.date = "{date}"
                """

    fund_df = spark.sql(query)
    fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
    
    return fund_df

def pull_prev_data(df, big_string, big_string2):
    df=spark.createDataFrame(df)
    df.createOrReplaceTempView("temp_table")
    
    query1 = f"""
                SELECT {big_string}
                FROM temp_table t  
                LEFT JOIN sym_ticker_region s ON t.Ticker = SUBSTRING(s.ticker_region, 1, LENGTH(s.ticker_region)-3) AND SUBSTRING(s.ticker_region, -2, 3) = 'US'
                LEFT JOIN FF_ADVANCED_QF a ON s.fsym_id = a.fsym_id AND YEAR(a.date) = YEAR(t.Implosion_Date) AND QUARTER(a.date) = QUARTER(t.Implosion_Date)
                ORDER BY t.Ticker
            """
    query2 = f"""
                SELECT {big_string2}
                FROM temp_table t  
                LEFT JOIN sym_ticker_region s ON t.Ticker = SUBSTRING(s.ticker_region, 1, LENGTH(s.ticker_region)-3) AND SUBSTRING(s.ticker_region, -2, 3) = 'US'
                LEFT JOIN FF_ADVANCED_QF b ON s.fsym_id = b.fsym_id AND YEAR(b.date) = YEAR(t.Implosion_Prev4Years) AND QUARTER(b.date) = QUARTER(t.Implosion_Prev4Years)
                ORDER BY t.Ticker
            """
    df1 = spark.sql(query1)
    df2 = spark.sql(query2)
    df1 = df1.toPandas()
    df2 = df2.toPandas()
    print(df1.head(5))
    print(df2.head(5))
    print(len(df1)==len(df2))
    percentage_change_df = ((df1 - df2) / df2) * 100
    print(percentage_change_df)
    
    # df['pct_change'] = (df[metric_curr] - df[metric_prev])/df[metric_prev]
    # df['pct_change'] = df['pct_change'].replace([np.inf, -np.inf], np.nan) 
    # df=df.dropna(axis=0)
    # mean_val = df['pct_change'].mean()
    # stddev_val = df['pct_change'].std()
    # z_score_threshold = 3.0
    # df = df[
    # (df['pct_change'] >= mean_val - z_score_threshold * stddev_val) &
    # (df['pct_change'] <= mean_val + z_score_threshold * stddev_val)]
    # new_mean = df['pct_change'].mean()
    # return new_mean


def create_df(filename):
    df = pd.read_csv(filename, index_col=False)
    df['Implosion_Date'] = pd.to_datetime(df['Implosion_Date']).dt.date
    df['Implosion_Year'] = df['Implosion_Date']
    df['Implosion_Year'] =  pd.to_datetime(df['Implosion_Year']).dt.year
    df['Implosion_Prev4Years'] = df['Implosion_Date'] - pd.DateOffset(years=1)
    df['Implosion_Prev4Years'] =  pd.to_datetime(df['Implosion_Prev4Years']).dt.year
    df = df.sort_values(by='Ticker')
    df_metrics = spark.sql("SELECT * FROM FF_BASIC_AF LIMIT 10")
    df_metrics = df_metrics.columns
    #df_metrics.remove('ff_actg_standard')
    df_metrics.remove('ff_eps_rpt_date')
    df_metrics.remove('ff_source_is_date')
    df_metrics.remove('ff_source_bs_date')
    df_metrics.remove('ff_source_cf_date')
    df_metrics.remove('ff_source_doc')
    df_metrics.remove('ff_report_freq_code')
    df_metrics.remove('ff_fiscal_date')
    df_metrics.remove('ff_actg_standard')
    df_metrics = df_metrics[6:100]
    metric_dict = {}
    for m in df_metrics:
        print(m)
        avg = pull_prev_data(df, m)
        metric_dict[m] = avg
    # df=pull_prev_data(df, 'FF_PRICE_CLOSE_FP')
    # return df
    metric_df = pd.DataFrame(list(metric_dict.items()), columns=['Metric', 'Value'])
    metric_df.to_csv('ChangesBeforeImplosion1yr.csv', index=False)
    return metric_df

def pull2(filename):
    df = pd.read_csv(filename, index_col=False)
    df['Implosion_Date'] = pd.to_datetime(df['Implosion_Date']).dt.date
    df['Implosion_Prev4Years'] = df['Implosion_Date'] - pd.DateOffset(years=4)
    df_metrics = spark.sql("SELECT * FROM FF_ADVANCED_QF LIMIT 10")
    df_metrics = df_metrics.columns
    df_metrics = df_metrics[5:7]
    result_string = ', '.join('a.' + item for item in df_metrics)
    result_string2 = ', '.join('b.' + item for item in df_metrics)
    #print(result_string)
    pull_prev_data(df, result_string, result_string2)

df = pull2('imploded_tickers_dates.csv')
# df.show(100)
# rdf=query('AAPL','2012-01-01')
# rdf.show()

   ff_amort_cf  ff_assets_sep_accts
0     0.011000                  NaN
1          NaN                  NaN
2          NaN                  NaN
3     7.807261                  NaN
4          NaN                  NaN
   ff_amort_cf  ff_assets_sep_accts
0        0.042                  NaN
1          NaN                  NaN
2          NaN                  NaN
3          NaN                  NaN
4          NaN                  NaN
True
     ff_amort_cf  ff_assets_sep_accts
0     -73.809524                  NaN
1            NaN                  NaN
2            NaN                  NaN
3            NaN                  NaN
4            NaN                  NaN
..           ...                  ...
473   261.323869                  NaN
474          NaN                  NaN
475          NaN                  NaN
476          NaN                  NaN
477          NaN                  NaN

[478 rows x 2 columns]


In [ ]:
#spark.stop()

In [19]:
def get_top_bottom_five(df):
    df = df.sort_values(by='Value')
    df=df.dropna()
    top5 = df.head(10)
    down5 = df.tail(10)
    print(top5)
    print(down5)


df = pd.read_csv('ChangesBeforeImplosion1yr.csv')
get_top_bottom_five(df)

               Metric       Value
8   ff_loan_loss_prov -152.202681
65     ff_for_exch_cf   -3.384404
55       ff_wkcap_chg   -1.504482
66     ff_chg_cash_cf   -1.243237
15      ff_eq_aff_inc   -0.797521
93      ff_rsrv_noneq   -0.750000
74  ff_price_close_fp   -0.669528
58    ff_loan_decr_cf   -0.631032
4     ff_oper_exp_oth   -0.599675
51      ff_dfd_tax_cf   -0.526569
                   Metric     Value
19           ff_cash_only  1.198095
41                ff_debt  1.310782
7          ff_int_exp_tot  1.440537
73         ff_com_shs_out  1.494861
92             ff_ppe_net  1.512333
31             ff_debt_st  1.572959
59  ff_sale_assets_bus_cf  2.036180
56          ff_acq_bus_cf  2.350508
26   ff_assets_oth_intang  6.881336
10       ff_ptx_xord_chrg  6.944790


In [ ]:
metric_df

In [ ]:
metric_dict